In [1]:
!pip install "giskard>=2.0.0b" -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.6/417.6 kB 18.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.1 kB/s eta 0:00:0000:0200:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.7 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 41.4 kB/s eta 0:00:0000:0100:14
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 63.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.5 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 42.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 45.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 37.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 48.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB

  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=dbd47b912afa1890439a56156793edbf14bd6999a33e1c7d47e39ff69ce50422
  Stored in directory: /home/usd.local/siddhi.bajracharya/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143860 sha256=921e7488a98c830f22aca84507010b5915030e451e355fdb53ef5df3591139cf
  Stored in directory: /home/usd.local/siddhi.bajracharya/.cache/pip/wheels/6b/97/66/9aea51f102b7bda562817122c900237a84f9f6649942879dd1
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=9fc53c7fe65eb7acc1dbb3dc5abdbc7bf76759ecf47db4a03e8facee31d733a2
  Stored in directory: /home/us

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torchaudio
import os
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
import librosa
import matplotlib.pyplot as plt
from torchaudio.utils import download_asset
import cv2
import glob
import pandas as pd


from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from torch import nn
import torch
import torch.optim as optim
import lightning.pytorch as pl
from torchaudio.transforms import *
import numpy as np
import torchmetrics
from lightning.pytorch.loggers import CSVLogger
from torch.nn.utils.rnn import pad_sequence
import re
from transformers import AutoFeatureExtractor, ASTForAudioClassification



In [3]:
llogger = CSVLogger("emotions", 'classification')

In [78]:
from torch.utils.data import Dataset
import glob

class AudioDataset(Dataset):
    """
        Custom dataset class for loading audio dataset.
        meta_df: dataframe containing file_name (without 
                the .wav extension) and the category (labels) 
        directory: regular expression for the directory to look 
                for wav files. (e.g. /dataset/speech/*.wav)
    """
    
    def __init__(self, meta_df, directory, **kwargs):
        self.meta_df = meta_df
        self.directory = directory
#         self.transforms = transforms
        self.audio_path_list = glob.glob(directory)
#         self.signal_trim = signal_trim
        category = self.meta_df['category'].unique()
        self.t_dict = dict(zip(category,range(len(category))))
        self.kwargs = kwargs
    
    def __len__(self):
        return len(self.audio_path_list)
    
    def __getitem__(self, idx):
#         t_dict = dict(zip(
#                 self.category,
#                     range(len(self.category))
#                 ))
        audio_path = self.audio_path_list[idx]
        audio_name = os.path.basename(audio_path).split('.')[0]
        targets = self.meta_df.loc[idx,[ 'category']].values[0]
        signal, sr = torchaudio.load(audio_path, )
        
        feature_extractor = AutoFeatureExtractor.from_pretrained("ast-finetuned-audioset-10-10-0.4593")
        
        signal =  signal[0]
        return feature_extractor(signal, sampling_rate=16000, return_tensors="pt"), self.t_dict[targets]
    

In [79]:
data_path = 'IEMOCAP_full_release/Session*/sentences/wav/Ses*/*.wav'
files = glob.glob(data_path)
file_name = [x.split(os.sep)[-1].split('.')[0] for x in files]

In [80]:
csv_file_paths = 'IEMOCAP_full_release/Session*/dialog/EmoEvaluation/Categorical/Ses*.txt'

In [81]:
csv_files = glob.glob(csv_file_paths)

In [82]:
def get_name_category_from_path(path):
    file_name_list = []
    emotion_list = []
    with open(path, 'r') as f:
        for line in f.readlines():
            file_name_list.append(re.findall('Ses.*[A-Z][0-9]{3}', line)[0])
            emotion_list.append(re.findall(':.*;', line)[0][1:-1].lower())
    return file_name_list, emotion_list
        

In [83]:
file_list = []
category_list = []
for file in csv_files:
    f, e = get_name_category_from_path(file)
    file_list.extend(f)
    category_list.extend(e)
meta_df = pd.DataFrame(
    {
        'file':file_list,
        'category':category_list
    }
)

In [84]:
meta_df['category'] = meta_df['category'].str.replace(';.*','')

In [85]:
meta_df['category'].value_counts()

frustration      7489
neutral state    6400
excited          4358
anger            3911
sadness          3521
happiness        2799
other             682
surprise          604
fear              260
disgust            93
Name: category, dtype: int64

In [86]:
meta_df['category'].unique()

array(['frustration', 'excited', 'sadness', 'fear', 'anger',
       'neutral state', 'happiness', 'surprise', 'disgust', 'other'],
      dtype=object)

In [87]:
asset = AudioDataset(meta_df,data_path)

In [88]:
asset.__getitem__(1)

({'input_values': tensor([[[-0.7030, -1.0522, -0.6753,  ..., -1.1763, -1.2540, -1.1932],
          [-0.8457, -1.2658, -0.8890,  ..., -1.2107, -1.2378, -1.1702],
          [-1.0629, -1.2776, -1.1097,  ..., -1.1437, -1.2557, -1.2330],
          ...,
          [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670],
          [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670],
          [ 0.4670,  0.4670,  0.4670,  ...,  0.4670,  0.4670,  0.4670]]])},
 0)

In [91]:

class LighteningModel(pl.LightningModule):
    def __init__(self, input_size=48000, num_classes=10, hidden_size = 200, num_heads = 5, num_layers_tx  = 2):
        super(LighteningModel, self).__init__()
        self.train_r2 = torchmetrics.R2Score()
        self.val_r2 = torchmetrics.R2Score()
        self.train_auc = torchmetrics.classification.AUROC(task = 'multiclass', num_classes=num_classes)
        self.val_auc = torchmetrics.classification.AUROC(task = 'multiclass', num_classes=num_classes)
        
        feature_extractor = AutoFeatureExtractor.from_pretrained("ast-finetuned-audioset-10-10-0.4593")
        self.sp_model = ASTForAudioClassification.from_pretrained("ast-finetuned-audioset-10-10-0.4593", return_dict=False)
        for param in self.sp_model.parameters():
            param.requires_grad = False
        self.sp_model.classifier.dense = nn.Linear(768,527)
        
        self.fc1 = nn.Linear(527, num_classes)
        self.dropout = nn.Dropout(.5) 
        self.relu = nn.ReLU()
    def forward(self,x,):
        x = x['input_values'].view(x['input_values'].size(0), 1024,128)
        
        out = self.sp_model(x, return_dict=False)[0] #256
        out = self.dropout(self.fc1(out))
#         out = self.sigmoid(out)
        return out
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.0001, weight_decay=1e-3)
        return optimizer
    
    
    def training_step(self, train_batch, batch_idx):
        signal, labels = train_batch
#         signal = signal['input_values'].view(signal['input_values'].size(0), 1024,128).shape
        labels =labels.float().to('cuda:0')
        outputs = self(signal).to('cuda:0')#.argmax(1).float()
        criterion = torch.nn.CrossEntropyLoss()
        loss = criterion(outputs, labels.long())
        self.train_auc(outputs, labels.int())
        self.log('loss', loss, on_step=False, on_epoch=True)
        self.log('train_auc', self.train_auc, on_step=False, on_epoch=True)
        
        
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        signal ,labels = val_batch
        labels = labels.float().to('cuda:0')
#         signal = signal['input_values'].view(signal['input_values'].size(0), 1024,128).shape
        
        outputs = self(signal).to('cuda:0')#.argmax(1).float()
        criterion = torch.nn.CrossEntropyLoss()
        loss = criterion(outputs, labels.long())
        self.val_auc(outputs, labels.int())
        
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_auc', self.val_auc, on_step=False, on_epoch=True)



In [92]:
model = LighteningModel()

In [93]:
next(model.parameters()).is_cuda

False

In [94]:
BATCH_SIZE = 32

In [95]:
dataset = AudioDataset(meta_df,data_path)

In [96]:
train_set, val_set = torch.utils.data.random_split(dataset, [0.8, 0.2],)

In [97]:
from torch.utils.data import DataLoader

In [98]:

from torch.utils.data import WeightedRandomSampler

In [99]:
# sampler = WeightedRandomSampler(sample_weights, int(len(train_set)*1.5), replacement=True)
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True, num_workers=8, drop_last=True, )
test_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, drop_last=True)

In [104]:
trainer = pl.Trainer( max_epochs=500, gradient_clip_val=0, logger = llogger, )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [105]:
trainer.fit(model, train_dataloader, test_dataloader, )

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                      | Params
--------------------------------------------------------
0 | train_r2  | R2Score                   | 0     
1 | val_r2    | R2Score                   | 0     
2 | train_auc | MulticlassAUROC           | 0     
3 | val_auc   | MulticlassAUROC           | 0     
4 | sp_model  | ASTForAudioClassification | 86.6 M
5 | fc1       | Linear                    | 5.3 K 
6 | dropout   | Dropout                   | 0     
7 | relu      | ReLU                      | 0     
--------------------------------------------------------
410 K     Trainable params
86.2 M    Non-trainable params
86.6 M    Total params
346.397   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [102]:
a = next(iter(train_dataloader))

In [ ]:
giskard_dataset = giskard.Dataset(
    df=train_loader,  # A pandas.DataFrame that contains the raw data (before all the pre-processing steps) and the actual ground truth variable (target).
    target="Survived",  # Ground truth variable
    name="Titanic dataset", # Optional
    cat_columns=['Pclass', 'Sex', "SibSp", "Parch", "Embarked"]  # Optional, but is a MUST if available. Inferred automatically if not.
)

In [ ]:
def prediction_function(df):
    return trainer.predi